# Webscrapper for Registro Público de Concesiones

Traditional webscrapping with Beautiful Soup is less useful for aspx websites that expect a user input. This webscraper was built with Selenium intenting to make requests to the form before scraping each page.

Following links are helpful: <br>
Data Source: https://avancedigital.mineco.gob.es/espectro/Paginas/registro-publico-concesiones.aspx <br>
Selenium: https://selenium-python.readthedocs.io/locating-elements.html <br>
Chromedriver: https://chromedriver.chromium.org/home (download necessary)<br>

In [180]:
#pip install requests (for making HTTP requests)
#pip install URLLib3 (URL handling)
#pip install bs4 (in case Selenium couldn’t handle everything)
#pip install selenium (for browser-based navigation)

## Loading libraries and preparing dataframe

In [181]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import math
import time

In [182]:
#Record execution time of script
start_time = time.time()

In [183]:
#Create a dataframe to store all of the scraped table data
df = pd.DataFrame(columns = ["Comunidad","Provincia","Municipio","Frecuencia"])

## Making crawler move on the website

In [184]:
# Establish chrome driver and go to report site URL
url = "https://sedeaplicaciones.minetur.gob.es/RPC_Consulta/FrmConsulta.aspx"
driver = webdriver.Chrome('C:/Users/jthoma/Downloads/chromedriver.exe') #use path where chormedriver saved
driver.get(url)

C:\Users\jthoma\AppData\Local\Temp\ipykernel_31580\1054195131.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/jthoma/Downloads/chromedriver.exe') #use path where chormedriver saved


## Scrapping the tables

- 02 ANDALUCÍA
- 03 ARAGÓN
- 04 CANARIAS
- 05 CANTABRIA
- 06 CASTILLA Y LEÓN
- 07 CASTILLA-LA MANCHA
- 08 CATALUÑA
- 09 CIUDAD AUTÓNOMA DE CEUTA
- 10 CIUDAD AUTÓNOMA DE MELILLA
- 11 COMUNIDAD DE MADRID
- 12 COMUNIDAD FORAL DE NAVARRA
- 13 COMUNITAT VALENCIANA
- 14 EXTREMADURA
- 15 GALICIA
- 16 ILLES BALEARS
- 17 LA RIOJA
- 18 PAÍS VASCO
- 19 PRINCIPADO DE ASTURIAS
- 20 REGIÓN DE MURCIA

In [185]:
total_concesiones = []

for comunidad in range(2,3): #21):
    
    #Select Servicio Fijo on Registro Público
    serviciofijo_select = driver.find_element(By.XPATH,'//*[@id="MainContent_rblTipoServicio_1"]')
    serviciofijo_select.click()
    
    #Select Comunidad on Registro Público
    comunidad_select = driver.find_element(By.XPATH,'//*[@id="MainContent_cmbComunidad"]/option['+str(comunidad)+']')
    comunidad_select.click()

    #Click on Buscar
    buscar_button = driver.find_element(By.XPATH,'//*[@id="MainContent_btnBuscar"]')
    buscar_button.click()
    driver.implicitly_wait(5) #Wait 5 seconds for the load
        
    #Getting the number of pages to scrap
    number_concesiones = driver.find_element(By.XPATH,'//*[@id="MainContent_lblTotal"]')
    total_concesiones.append(int(str.split(number_concesiones.text)[0]))
    pages = math.ceil((int(str.split(number_concesiones.text)[0]))/10)

    
    #Flip through all of the records and save them
    for n in range(2,4): #pages+1):
        
        #Click on each Referencia link
        for link in range(0,9):
            referencia_link = driver.find_element(By.XPATH,'//*[@id="MainContent_gridConcesiones_lnkFrecuencias_'+str(link)+'"]')
            referencia_link.click()
        
        
            for i in range(3):
                try:
                    mytable = driver.find_element(By.XPATH,'//*[@id="MainContent_gridFrecuencias"]')
                    #Read in all of the data into the dataframe
                    for row in mytable.find_elements(By.CSS_SELECTOR,'tr'):
                        row_list = []
                        #Add to dataframe accordingly
                        for cell in row.find_elements(By.CSS_SELECTOR,'td'):
                            cell_reading = cell.text
                            row_list.append(cell_reading)
                        #Add the list as a row, if possible 
                        try:
                            a_series = pd.Series(row_list, index = df.columns)
                            df = df.append(a_series, ignore_index=True)
                        except:
                            print("Could not append: " + str(row_list))
                    break
                except:
                    driver.implicitly_wait(1)
                
                
                #Click on Cerrar    
                try:
                    cerrar_button = driver.find_element(By.XPATH,'//*[@id="MainContent_btnCerrar"]')
                    cerrar_button.click()
                except:
                    cerrar_button1 = driver.find_element(By.CLASS_NAME,'botonbuscar')
                    driver.execute_script("arguments[0].click();", cerrar_button1)
                driver.implicitly_wait(3)
            
            pass
    
        if n%10 == 1:
            #Click second "..." if on greater than page 10
            if n < 20:
                driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[11]/a')[0].click()  
            else:
                driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[12]/a')[0].click()  
        else:
            driver.find_element(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[' + str(n)+ ']').click() 
        #Wait three seconds so the website doesn't crash
        driver.implicitly_wait(3)

    #if n%10 == 1:
     #   #Click second "..." if on greater than page 10
      #  if n < 20:
       #     driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[11]/a')[0].click()  
        #else:
         #   driver.find_elements(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[12]/a')[0].click()  
    #else:
     #   driver.find_element(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[' + str(n)+ ']').click()    
    #Wait three seconds so the website doesn't crash
    #driver.implicitly_wait(4)
        
    
    #go back to selecting comunidad
    nueva_busqueda = driver.find_element(By.XPATH,'//*[@id="MainContent_btnNuevaBusqueda"]')
    nueva_busqueda.click()
    driver.implicitly_wait(5) #Wait 5 seconds for the load

C:\Users\jthoma\AppData\Local\Temp\ipykernel_31580\3841225853.py:45: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  a_series = pd.Series(row_list, index = df.columns)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_31580\3841225853.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)
C:\Users\jthoma\AppData\Local\Temp\ipykernel_31580\3841225853.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a_series, ignore_index=True)


Could not append: []


ElementClickInterceptedException: Message: element click intercepted: Element <a id="MainContent_gridConcesiones_lnkFrecuencias_1" title="Ver detalles de las frecuencias de uso" href="javascript:__doPostBack('ctl00$MainContent$gridConcesiones$ctl03$lnkFrecuencias','')" tabindex="-1">...</a> is not clickable at point (397, 442). Other element would receive the click: <div id="MainContent_modalCesionTransferencia_backgroundElement" class="modalBackground" style="position: fixed; left: 0px; top: 0px; z-index: 10000; width: 1903px; height: 1059px;"></div>
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x01176463+2188387]
	Ordinal0 [0x0110E461+1762401]
	Ordinal0 [0x01023D78+802168]
	Ordinal0 [0x01057F9B+1015707]
	Ordinal0 [0x01055F68+1007464]
	Ordinal0 [0x01053C6B+998507]
	Ordinal0 [0x010529D9+993753]
	Ordinal0 [0x01048613+951827]
	Ordinal0 [0x0106C7DC+1099740]
	Ordinal0 [0x01047FF4+950260]
	Ordinal0 [0x0106C9F4+1100276]
	Ordinal0 [0x0107CC22+1166370]
	Ordinal0 [0

In [ ]:
df

In [ ]:
#Write to a csv
df.to_csv("RegistroPublicoConcesiones_Frequencies1.csv", index= False,sep=";")

In [ ]:
sum(total_concesiones)

In [ ]:
print("--- %s minutes ---" % ((time.time() - start_time)/60))

driver.find_element(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[' + str(4)+ ']').click()

In [ ]:
driver.find_element(By.XPATH,'//*[@id="MainContent_gridConcesiones"]/tbody/tr[12]/td/table/tbody/tr/td[' + str(n)+ ']').click()